<a href="https://colab.research.google.com/github/obeabi/Sentiment-Labelled-Sentences-Data-Set/blob/main/Main2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment140 - A Twitter Sentiment Analysis Tool
# Written by Abiola Obembe
## Date: 2020-12-


Abstract
The data is a CSV with emoticons removed. Data file format has 6 fields:
0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
1 - the id of the tweet (2087)
2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)
3 - the query (lyx). If there is no query, then this value is NO_QUERY.
4 - the user that tweeted (robotickilldozr)
5 - the text of the tweet (Lyx is cool)

## Step 1: Import dependencies

In [9]:
#Import libraries
import numpy as np
import pandas as pd
import re
import math
from bs4 import BeautifulSoup


# Tensorflow v2

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
print("The tensorflow version is :", tf.__version__)


print("Dependencies installed succesffuly!")

The tensorflow version is : 2.3.0
Dependencies installed succesffuly!



## Step 2: Data Preprocessing

In [15]:
# Load data files
cols = ["sentiment", "id", "date", "query", "user", "text"]

train_data = pd.read_csv("trainingdata.csv",error_bad_lines=False,
    header=None,names=cols,engine="python",encoding="latin1")

test_data = pd.read_csv(
    "testdata.csv",error_bad_lines=False,
    header=None, names=cols,engine="python",encoding="latin1")


print("Training set :", train_data.shape )
print("Test set :", test_data.shape)

train_data.head()

Skipping line 203629: unexpected end of data


Training set : (203628, 6)
Test set : (498, 6)


,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
# clean data
train_data.drop(['id','date', 'query','user'], axis = 1, inplace= True)
train_data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [12]:
# Examine sentiment data labels
train_data['sentiment'].value_counts()

0    260035
Name: sentiment, dtype: int64

In [13]:
# Function to clean
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    tweet = tweet.lower()
    
    return tweet

In [14]:
# apply function on dataset
data_train_clean = [ clean_tweet(tweet)  for tweet in train_data.text]

In [ ]:
# Evaluate the sentiment column
set(train_data.sentiment.values)

In [ ]:
# Let's set the sentiment values labelled 4 as 1
data_labels = train_data.sentiment.values
data_labels[data_labels == 4] = 1
set(data_labels)

In [ ]:
# Tokenization
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences

